## Bringing together events & activities

hours: pivot by available dates
events: get lat/lon

In [45]:
#!pip install geopy

In [46]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm

pd.set_option('display.max_rows', 4000)

In [47]:
places = pd.read_csv("bay_full.csv")
events = pd.read_csv("events.csv")

In [48]:
events.head()

,Unnamed: 0,date,time,title,image_link,address,event_url,continuous
0,0,19-04-2024,8:00PM,The Speakeasy: Age of Scofflaws,'https://assets0.dostuffmedia.com/uploads/aws_...,"644 Broadway, San Francisco",http://sf-events.brokeassstuart.com/events/202...,Through Jun 23
1,1,19-04-2024,10:00AM,Week of 420 at The Apothecarium,'https://assets0.dostuffmedia.com/uploads/aws_...,2029 Market St,http://sf-events.brokeassstuart.com/events/202...,Through Apr 20
2,2,19-04-2024,9:00AM,Discovery Sessions 2024,'https://assets0.dostuffmedia.com/uploads/aws_...,900 Marin St.,http://sf-events.brokeassstuart.com/events/202...,Through Apr 19
3,3,19-04-2024,9:30AM,Irving Penn,'https://assets0.dostuffmedia.com/uploads/aws_...,50 Hagiwara Tea Garden Drive,http://sf-events.brokeassstuart.com/events/202...,Through Jul 21
4,4,19-04-2024,9:00PM,Still Corners,'https://assets0.dostuffmedia.com/uploads/aws_...,859 O'farrell Street,http://sf-events.brokeassstuart.com/events/202...,NaN


In [49]:
aggregated = events.groupby('title').agg({
    'date': list,
    'time': 'first',
    'address': 'first',
    'event_url': 'first'
}).reset_index()
aggregated.rename(columns={'event_url': 'link'}, inplace=True)

In [50]:
aggregated

,title,date,time,address,link
0,NightZ @ Z,[25-04-2024],6:00PM,450 Florida Street,http://sf-events.brokeassstuart.com/events/202...
1,"""ordinary. extraordinary."" Artist Talk April 2...",[20-04-2024],4:00PM,1339 61st St.,http://sf-events.brokeassstuart.com/events/202...
2,2024 Bay Area Brew Fest,[20-04-2024],12:00PM,42 Moraga Avenue,http://sf-events.brokeassstuart.com/events/202...
3,2024 Bay Area Harmony Sweepstakes A Cappella F...,[27-04-2024],8:00PM,3301 Lyon Street,http://sf-events.brokeassstuart.com/events/202...
4,2024 Founder’s Concert,[18-05-2024],3:00PM,2727 College Ave,http://sf-events.brokeassstuart.com/events/202...
5,2024 King's Day,[27-04-2024],12:00PM,None,http://sf-events.brokeassstuart.com/events/202...
6,3rd Annual Celebration Of Prince With Graffiti...,[21-04-2024],7:00PM,None,http://sf-events.brokeassstuart.com/events/202...
7,420 Brunch w/Outlaw & Allard,[20-04-2024],11:00AM,"803 Wright Avenue, Suite 101",http://sf-events.brokeassstuart.com/events/202...
8,420 Experience with Mistah Fab and Cuzzin Dre,[20-04-2024],12:00PM,2019 MacArthur Blvd,http://sf-events.brokeassstuart.com/events/202...
9,67th San Francisco International Film Festival,"[24-04-2024, 25-04-2024, 26-04-2024, 27-04-202...",7:00PM,None,http://sf-events.brokeassstuart.com/events/202...


In [51]:
"""
# Works!

geolocator = Nominatim(user_agent="neural-navigator")
your_loc = input("Enter your location: ")
location = geolocator.geocode(your_loc).raw
"""

### WARNING: THIS TAKES A WHILE, DO NOT ACCIDENTALLY RUN THIS CELL AND OVERRIDE PROGRESS

geolocator = Nominatim(user_agent="neural-navigator")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

for index, row in tqdm(aggregated.iterrows()):
    if row["address"]:
        address = row["address"].replace(".", "").lower() + ", san francisco"
        geolocation = geocode(address)
        if geolocation:
            aggregated.at[index, "address"] = geolocation.address
            aggregated.at[index, "latitude"] = geolocation.latitude
            aggregated.at[index, "longitude"] = geolocation.longitude
        else:
            print(f"Could not resolve address: {address}")

#events['location'] = events['address'].head(1).apply(geocode)
#events['latitude'] = events['location'].apply(lambda loc: loc.latitude if loc else None)
#events['longitude'] = events['location'].apply(lambda loc: loc.longitude if loc else None)

2it [00:01,  1.02it/s]

Could not resolve address: 1339 61st st, san francisco


5it [00:04,  1.12s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('803 wright avenue, suite 101, san francisco',), **{}).
Traceback (most recent call last):
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 1349, in getresponse
    response.begin()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Version

Could not resolve address: 803 wright avenue, suite 101, san francisco


23it [00:38,  1.12s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('400 s 1st street, san francisco',), **{}).
Traceback (most recent call last):
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 1349, in getresponse
    response.begin()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/p

...

RateLimiter caught an error, retrying (0/2 tries). Called with (*('buchanan st and marina st, san francisco',), **{}).
Traceback (most recent call last):
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 1349, in getresponse
    response.begin()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/

Could not resolve address: buchanan st and marina st, san francisco


489it [16:20,  4.95s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('101 sixth st, san francisco',), **{}).
Traceback (most recent call last):
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/Users/cyprianzander/Desktop/Neural Navigator/neuralnav/lib/python3.9/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 1349, in getresponse
    response.begin()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/pyth

Could not resolve address: 101 sixth st, san francisco


492it [16:42,  2.04s/it]


In [52]:
aggregated.head(500)

,title,date,time,address,link,latitude,longitude
0,NightZ @ Z,[25-04-2024],6:00PM,"Z Space, 450, Florida Street, Mission District...",http://sf-events.brokeassstuart.com/events/202...,37.763455,-122.411437
1,"""ordinary. extraordinary."" Artist Talk April 2...",[20-04-2024],4:00PM,1339 61st St.,http://sf-events.brokeassstuart.com/events/202...,NaN,NaN
2,2024 Bay Area Brew Fest,[20-04-2024],12:00PM,"Inn at the Presidio, 42, Moraga Avenue, San Fr...",http://sf-events.brokeassstuart.com/events/202...,37.797066,-122.457846
3,2024 Bay Area Harmony Sweepstakes A Cappella F...,[27-04-2024],8:00PM,"Palace of Fine Arts, 3301, Lyon Street, San Fr...",http://sf-events.brokeassstuart.com/events/202...,37.802919,-122.448403
4,2024 Founder’s Concert,[18-05-2024],3:00PM,"College Avenue, St. Mary's Park, San Francisco...",http://sf-events.brokeassstuart.com/events/202...,37.733580,-122.423753
5,2024 King's Day,[27-04-2024],12:00PM,None,http://sf-events.brokeassstuart.com/events/202...,NaN,NaN
6,3rd Annual Celebration Of Prince With Graffiti...,[21-04-2024],7:00PM,None,http://sf-events.brokeassstuart.com/events/202...,NaN,NaN
7,420 Brunch w/Outlaw & Allard,[20-04-2024],11:00AM,"803 Wright Avenue, Suite 101",http://sf-events.brokeassstuart.com/events/202...,NaN,NaN
8,420 Experience with Mistah Fab and Cuzzin Dre,[20-04-2024],12:00PM,"General Douglas MacArthur Tunnel, San Francisc...",http://sf-events.brokeassstuart.com/events/202...,37.792589,-122.469657
9,67th San Francisco International Film Festival,"[24-04-2024, 25-04-2024, 26-04-2024, 27-04-202...",7:00PM,None,http://sf-events.brokeassstuart.com/events/202...,NaN,NaN


In [53]:
aggregated.to_csv("events_with_locations_2.csv")

#### Location open hours fixing

In [19]:
import json
import numpy as np

In [20]:
bay_full = pd.read_csv("bay_full.csv")
bay_comprehensive = pd.read_csv("comprehensive_berkeley.csv")

In [21]:
bay_comprehensive = bay_comprehensive[["title", "link", "open_hours"]]

In [22]:
for index, row in bay_comprehensive.iterrows():
    open_hours = row["open_hours"]
    open_hours_parsed = json.loads(open_hours)
    new_open_hours = []
    dow_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    if len(open_hours_parsed.keys()) == 7:
        for dow in dow_order:
            bay_comprehensive.at[index, dow.lower()] = open_hours_parsed[dow][0]

In [23]:
bay_comprehensive = bay_comprehensive.drop("open_hours", axis=1)
bay_comprehensive.head()

,title,link,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Alameda Aero Club,https://www.google.com/maps/place/Alameda+Aero...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Big C,https://www.google.com/maps/place/Big+C/data=!...,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours,Open 24 hours
2,NRI Flying Club,https://www.google.com/maps/place/NRI+Flying+C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shorebird Park Nature Center,https://www.google.com/maps/place/Shorebird+Pa...,Closed,9 AM–5 PM,9 AM–5 PM,9 AM–5 PM,9 AM–5 PM,9 AM–5 PM,Closed
4,Berkeley Rose Garden,https://www.google.com/maps/place/Berkeley+Ros...,6 AM–10 PM,6 AM–10 PM,6 AM–10 PM,6 AM–10 PM,6 AM–10 PM,6 AM–10 PM,6 AM–10 PM


In [24]:
print(bay_full['link'].duplicated().sum())
print(bay_comprehensive['link'].duplicated().sum())

bay_comprehensive = bay_comprehensive.drop_duplicates(subset='link', keep='first')

print(bay_comprehensive['link'].duplicated().sum())

0
309
0


In [25]:
combined = pd.merge(bay_full, bay_comprehensive, on='title', how='left')

In [26]:
print([len(bay_full), len(bay_comprehensive), len(combined)])
combined.head()

[826, 601, 827]


,Unnamed: 0,title,features,quality,price,popularity,category,latitude,longitude,address,link_x,link_y,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,0,La Note,"{'brunch': 185, 'pancakes': 96, 'french toast'...",4.4,2,1702,French restaurant,37.866206,-122.267249,"2377 Shattuck Ave., Berkeley, CA 94704",https://www.google.com/maps/place/La+Note/data...,https://www.google.com/maps/place/La+Note/data...,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–3 PM,8 AM–3 PM
1,1,Grégoire Restaurant,"{'puffs': 115, 'fried chicken sandwich': 42, '...",4.5,2,917,French restaurant,37.878583,-122.268578,"2109 Cedar St, Berkeley, CA 94709",https://www.google.com/maps/place/Gr%C3%A9goir...,https://www.google.com/maps/place/Gr%C3%A9goir...,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM
2,2,À Côté,"{'mussels': 28, 'wine list': 19, 'cocktails': ...",4.4,3,413,Restaurant,37.842073,-122.251264,"5478 College Ave, Oakland, CA 94618",https://www.google.com/maps/place/%C3%80+C%C3%...,https://www.google.com/maps/place/%C3%80+C%C3%...,Closed,5–9 PM,5–9 PM,5–9 PM,5–10 PM,5–10 PM,Closed
3,3,Julia's Restaurant,"{'architecture': 5, 'hotel': 4, 'wine': 4, 'st...",4.4,0,102,French restaurant,37.867621,-122.262837,"2315 Durant Ave 2nd floor, Berkeley, CA 94704",https://www.google.com/maps/place/Julia's+Rest...,https://www.google.com/maps/place/Julia's+Rest...,Closed,12–2 PM,12–2 PM,12–2 PM,12–2 PM,12–2 PM,Closed
4,4,Le Bateau Ivre Restaurant,"{'atmosphere': 21, 'coffee': 16, 'brunch': 13,...",4.3,2,241,Restaurant,37.862503,-122.258559,"2629 Telegraph Ave, Berkeley, CA 94704",https://www.google.com/maps/place/Le+Bateau+Iv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
combined = combined.drop(columns=["Unnamed: 0", "quality", "price", "popularity", "features", "link_y"])

In [28]:
combined.rename(columns={'link_x': 'link'}, inplace=True)

In [29]:
combined

,title,category,latitude,longitude,address,link,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,La Note,French restaurant,37.866206,-122.267249,"2377 Shattuck Ave., Berkeley, CA 94704",https://www.google.com/maps/place/La+Note/data...,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–3 PM,8 AM–3 PM
1,Grégoire Restaurant,French restaurant,37.878583,-122.268578,"2109 Cedar St, Berkeley, CA 94709",https://www.google.com/maps/place/Gr%C3%A9goir...,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM
2,À Côté,Restaurant,37.842073,-122.251264,"5478 College Ave, Oakland, CA 94618",https://www.google.com/maps/place/%C3%80+C%C3%...,Closed,5–9 PM,5–9 PM,5–9 PM,5–10 PM,5–10 PM,Closed
3,Julia's Restaurant,French restaurant,37.867621,-122.262837,"2315 Durant Ave 2nd floor, Berkeley, CA 94704",https://www.google.com/maps/place/Julia's+Rest...,Closed,12–2 PM,12–2 PM,12–2 PM,12–2 PM,12–2 PM,Closed
4,Le Bateau Ivre Restaurant,Restaurant,37.862503,-122.258559,"2629 Telegraph Ave, Berkeley, CA 94704",https://www.google.com/maps/place/Le+Bateau+Iv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The Butcher’s Son Vegan Delicatessen & Bakery,Vegan restaurant,37.871575,-122.271629,"1954 University Ave, Berkeley, CA 94704",https://www.google.com/maps/place/The+Butcher%...,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,10 AM–8 PM,10 AM–4 PM
6,Tane Vegan Izakaya,Vegan restaurant,37.872754,-122.268800,"1956 Shattuck Ave., Berkeley, CA 94704",https://www.google.com/maps/place/Tane+Vegan+I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Humbowl,Health food restaurant,37.856681,-122.252904,"2975 College Ave, Berkeley, CA 94705",https://www.google.com/maps/place/Humbowl/data...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FAVA,Lunch restaurant,37.880211,-122.268634,"2114 Vine St, Berkeley, CA 94709",https://www.google.com/maps/place/FAVA/data=!4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Long Life Vegi House,Vegetarian restaurant,37.871280,-122.276776,"1725 University Ave, Berkeley, CA 94703",https://www.google.com/maps/place/Long+Life+Ve...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
new_columns = ['date', 'time']

for column in new_columns:
    combined[column] = np.nan

In [31]:
combined

,title,category,latitude,longitude,address,link,monday,tuesday,wednesday,thursday,friday,saturday,sunday,date,time
0,La Note,French restaurant,37.866206,-122.267249,"2377 Shattuck Ave., Berkeley, CA 94704",https://www.google.com/maps/place/La+Note/data...,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–2 PM,8 AM–3 PM,8 AM–3 PM,NaN,NaN
1,Grégoire Restaurant,French restaurant,37.878583,-122.268578,"2109 Cedar St, Berkeley, CA 94709",https://www.google.com/maps/place/Gr%C3%A9goir...,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,11:30 AM–7 PM,NaN,NaN
2,À Côté,Restaurant,37.842073,-122.251264,"5478 College Ave, Oakland, CA 94618",https://www.google.com/maps/place/%C3%80+C%C3%...,Closed,5–9 PM,5–9 PM,5–9 PM,5–10 PM,5–10 PM,Closed,NaN,NaN
3,Julia's Restaurant,French restaurant,37.867621,-122.262837,"2315 Durant Ave 2nd floor, Berkeley, CA 94704",https://www.google.com/maps/place/Julia's+Rest...,Closed,12–2 PM,12–2 PM,12–2 PM,12–2 PM,12–2 PM,Closed,NaN,NaN
4,Le Bateau Ivre Restaurant,Restaurant,37.862503,-122.258559,"2629 Telegraph Ave, Berkeley, CA 94704",https://www.google.com/maps/place/Le+Bateau+Iv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The Butcher’s Son Vegan Delicatessen & Bakery,Vegan restaurant,37.871575,-122.271629,"1954 University Ave, Berkeley, CA 94704",https://www.google.com/maps/place/The+Butcher%...,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,11 AM–8 PM,10 AM–8 PM,10 AM–4 PM,NaN,NaN
6,Tane Vegan Izakaya,Vegan restaurant,37.872754,-122.268800,"1956 Shattuck Ave., Berkeley, CA 94704",https://www.google.com/maps/place/Tane+Vegan+I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Humbowl,Health food restaurant,37.856681,-122.252904,"2975 College Ave, Berkeley, CA 94705",https://www.google.com/maps/place/Humbowl/data...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FAVA,Lunch restaurant,37.880211,-122.268634,"2114 Vine St, Berkeley, CA 94709",https://www.google.com/maps/place/FAVA/data=!4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Long Life Vegi House,Vegetarian restaurant,37.871280,-122.276776,"1725 University Ave, Berkeley, CA 94703",https://www.google.com/maps/place/Long+Life+Ve...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
combined.to_csv("activities_with_times.csv")